In [11]:
!pip install dicomsdl

In [12]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import tqdm
import dicomsdl as dicom


In [ ]:
train = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
test = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
train.head()
train = train[[col for col in train.columns if col in ['site_id',
 'patient_id',
 'image_id',
 'laterality',
 'view',
 'age',
 'implant',
 'machine_id',
 'prediction_id','cancer']]]
test.drop(['prediction_id'],axis=1,inplace=True)
train.loc[train['age'].isnull(),'age'] = train['age'].mean()

#loading directory of images
path = os.path.join('/kaggle/input/rsna-breast-cancer-detection/train_images')
list_dir = os.listdir(path)
k =[]
y={}
path_list=[]
lat_list = []
for i, sub in tqdm.tqdm(enumerate(list_dir)):
    for j, fname in enumerate(os.listdir(os.path.join(path,sub))):
        view = (train.loc[train['image_id']==int(((fname).split('.'))[0]),'view'].values)[0]
        laterality = (train.loc[train['image_id']==int(((fname).split('.'))[0]),'laterality'].values)[0]
        cancer = int(train.loc[train['image_id']==int(((fname).split('.'))[0]),'cancer'])
        k.append(os.path.join(os.path.join(path,sub),fname))
        y[int(((fname).split('.'))[0])] = cancer
        lat_list.append(laterality)
        if view =='MLO':
            if laterality=='L':
                 #mlo_path_l.append(os.path.join(os.path.join(path,sub),fname)
                 path_list.append(os.path.join(os.path.join(path,sub),fname))
            else:
                #mlo_path_r.append(os.path.join(os.path.join(path,sub),fname))
                path_list.append(os.path.join(os.path.join(path,sub),fname))
        #elif view =='CC':
         #   if laterality=='L':
         #       #cc_path_l.append(os.path.join(os.path.join(path,sub),fname))
         #       path_list.append(os.path.join(os.path.join(path,sub),fname))
         #   else:
         #       #cc_path_r.append(os.path.join(os.path.join(path,sub),fname))
          #      path_list.append(os.path.join(os.path.join(path,sub),fname))
path_dict={}
for i in range(len(list(path_list))):
    path_dict[i] = path_list[i]
lat_dict={}
for i in range(len(list(lat_list))):
    lat_dict[i] = lat_list[i]    
                                 


5708it [00:40, 123.86it/s]

In [ ]:
tf.config.run_functions_eagerly(True)
@tf.function
def gen(x):
    i = path_dict[x]
    l = lat_dict[x]
    image = (dicom.open(i).pixelData(storedvalue=True))
    max_val = tf.cast(image[-1,1000],dtype=tf.float32)
    label = y[int(i.split('/')[-1].split('.')[0])]
    #print(max_val)
    #if  train[train['image_id']==int(i.split('/')[-1].split('.')[0])]['laterality'].values[0]=='L':
    if l =='L':
        if max_val>=1000:
            return max_val-tf.image.resize(tf.expand_dims(image[100:-100,:3500],axis=-1),[150,100],preserve_aspect_ratio=False)/255.0,tf.convert_to_tensor(label)
        else:
            return tf.image.resize(tf.expand_dims(image[100:-100,:3500],axis=-1),[150,100],preserve_aspect_ratio=False)/255.0,tf.convert_to_tensor(label)
    else:
        if max_val>=1000:
            return max_val-tf.image.resize(tf.image.flip_left_right(tf.expand_dims(image,axis=-1))[100:-100,:3500,:],[150,100],preserve_aspect_ratio=False)/255.0,tf.convert_to_tensor(label)
        else:
            return tf.image.resize(tf.image.flip_left_right(tf.expand_dims(image,axis=-1))[100:-100,:3500,:],[150,100],preserve_aspect_ratio=False)/255.0,tf.convert_to_tensor(label)
    #elif train[train['image_id']==int(i.split('/')[-1].split('.')[0])]['laterality'].values[0]=='R':
      #  if max_val>=1000:
      #      return ((max_val-tf.image.resize_with_pad(tf.image.flip_left_right(tf.expand_dims(image,axis=-1)),500,500))/255.0)[:,140:-60,0],(y[int(i.split('/')[-1].split('.')[0])])
      #  else:
       #     return (tf.image.resize_with_pad(tf.image.flip_left_right(tf.expand_dims(image,axis=-1)),500,500)/255.0)[:,140:-60,0],(y[int(i.split('/')[-1].split('.')[0])])


tf.image.per_image_standardization

# Create a TensorFlow dataset and map your function to it
dataset= tf.data.Dataset.from_tensor_slices(np.arange((len(list(path_list[:1600]))))).prefetch(tf.data.AUTOTUNE).shuffle(2000)            
train_dataset = dataset.map(lambda x: tf.numpy_function(gen, inp=[x], Tout=(tf.float32,tf.int32))).batch(16)      

for i, j in train_dataset.take(16):
    print(i.shape)

In [ ]:
def pfbeta_fast(labels, predictions, beta=1):

    pTP = tf.math.reduce_sum(labels * predictions)
    pFP = tf.math.reduce_sum((1-labels) * predictions)
    num_positives = tf.math.reduce_sum((labels))  #  = pTP+pFN

    pPrecision = pTP/(pTP+pFP)
    pRecall = pTP/num_positives

    beta_squared = beta**2

    if (pPrecision > 0 and pRecall > 0):
        pF1 = (1+beta_squared) * pPrecision * pRecall/(beta_squared*pPrecision + pRecall)
        return pF1
    else:
        return 0.0
def cnn():
    
    
    pre = tf.keras.applications.convnext.ConvNeXtSmall(
    model_name='convnext_small',
    include_top=False,
    include_preprocessing=True,
    weights='imagenet',
    input_shape=(150,100,3),pooling='avg')
    
    pre.trainable=True
    
    for layer in pre.layers[:-10]:
        layer.trainable=False
    inputs = tf.keras.Input(shape=(150,100,1))
    X = tf.keras.layers.Concatenate()([inputs for i in range(3)])
    X = pre(X)
    #X = tf.keras.layers.Dropout(0.20)(X)
    outputs = tf.keras.layers.Dense(units=1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=inputs,outputs=outputs)
    model.summary()
    model.compile(loss='BinaryCrossentropy',optimizer = tf.keras.optimizers.Adam(),metrics=['accuracy',pfbeta_fast])
    model.fit(train_dataset,verbose=True,epochs=3,callbacks=[tf.keras.callbacks.ModelCheckpoint('model.h5')])
    return model



In [ ]:
tf.keras.backend.clear_session()
cnn()
 